# Vision

A significant part of this project relies on computer vision. We made extensive use of the *OpenCV* library to implement the image-processing pipeline that enables the robot to interpret its environment. All vision-related functionalities are organized within a dedicated `VisionSystem` class, implemented in the file *vision_system.py*.

The camera was placed above the setup at a slight angle. As a first step, we calibrated the image by detecting ArUco markers positioned around the environment. These markers allowed us to isolate and crop the relevant rectangular area. We then applied a perspective transform to obtain a top-down, orthonormal view of the scene, which became the reference frame for all subsequent processing. ArUco markers were also used to detect both the robot and the goal position.

To detect obstacles, we initially considered using black objects combined with classical edge detection. However, the presence of other dark elements in the scene such as the ArUco markers and parts of the Thymio robot made this approach unreliable. We therefore switched to using blue obstacles and applied color filtering to extract blue regions from the image. This method turned out to be highly effective and was ultimately adopted in our final pipeline.

METTRE PHOTO DU SETUP

At the beginning of the project, we planned to use the provided Aukey webcam. However, once we started coding and sharing our programs, we quickly ran into practical issues since we only had a single webcam. As we all work on Mac, a much more efficient solution was to use our iPhones as cameras instead. Thanks to the Apple ecosystem, each of us could easily use our iPhone camera as an external webcam for our MacBooks. This gave us higher image quality and allowed everyone to run and test the vision code independently, without relying on the one shared webcam.

We began by implementing the `VisionSystem` constructor, which initializes all required variables. It opens the camera stream using OpenCV’s `VideoCapture` and loads the ArUco dictionary.

In [ ]:
import cv2
import numpy as np
from cv2 import aruco

class VisionSystem:

    def __init__(self, camera_id=0, aruco_dict_type=aruco.DICT_4X4_50):
        self.cap = cv2.VideoCapture(camera_id)
        self.aruco_dict = aruco.getPredefinedDictionary(aruco_dict_type)
        self.aruco_params = aruco.DetectorParameters()

        # Initialize variables
        self.detected_markers = {}
        self.corners = None  # [(x1, y1), ...]
        self.transform_matrix = None
        self.mm2px = None
        self.map_size = (900, 600)
        self.goal_position = None  # (x, y)

The core of the setup is the `calibrate` function, which reads frames from the camera and detects all relevant ArUco markers. To differentiate their roles, we pass two parameters: `corner_ids` for the four map corners and `goal_id` for the goal marker.
Once all four corner markers are detected, we order them, draw their positions in green on the displayed image, and connect them with green lines to visualize the rectangular map region that will be extracted. We also highlight the goal marker with a circle and label it “GOAL”.


METTRE SCREEN DES LIGNES VERTES

When the user is satisfied with the setup and marker detection, pressing the “c” key completes the calibration. This triggers the perspective transform and computes the millimeter-to-pixel scaling factor used throughout the rest of the project.

METTRE SCREEN DE L’AFFICHAGE APRES C

### Converting Real-World Distances (mm) to Pixels

During calibration, the physical height of the map is known:

$H_{\text{real}}$ (in mm)

and the corresponding height in the transformed top-down image is fixed:

$H_{\text{px}}$ (in pixels)

The conversion factor from millimeters to pixels is therefore:
$$ \text{mm2px} = \frac{H_{\text{px}}}{H_{\text{real}}} $$

This scalar tells us how many pixels correspond to one millimeter in the real world.  
It allows converting any real-world measurement $d_{\text{mm}}$ into pixel units:
$$ d_{\text{px}} = d_{\text{mm}} \cdot \text{mm2px} $$


### Robot pose detection

We use the function `detect_raw_robot_pose` to detect the robot pose, and get its orientation. To do this, we use trigonometry. Here is the thought process behind the robot orientation estimation from the aruco marker :

In [ ]:
def detect_robot_raw_pose(self, frame):
    """
    Returns:
        np.array: [x, y, theta] in map coordinates, or None if not detected
    """

    marker_centers = self._detect_marker_centers(frame, target_ids={4})

    if 4 not in marker_centers:
        print("Robot marker not detected!")
        return None

    robot_data = self.detected_markers[4]

    corners = robot_data['corners'] # Orientation, corners are: TL, TR, BR, BL

    tl = corners[0]
    tr = corners[1]

    top_center = (tl + tr) / 2
    center = robot_data['center']
    x, y = center

    dx = top_center[0] - center[0]
    dy = top_center[1] - center[1]

    theta = np.arctan2(dy, dx) # Compute angle in radians

    return np.array([x, y, theta])

To compute the robot’s orientation, we use the top edge of the ArUco marker as a reference direction.

Given the marker corners TL and TR, and the marker center c, we define:

$$
m = \frac{TL + TR}{2}
$$

$$
v = m - c
$$

Component-wise:

$$
v_x = m_x - c_x, \qquad v_y = m_y - c_y
$$

The robot’s orientation angle is then obtained using:

$$
\theta = \operatorname{atan2}(v_y, v_x)
$$

This angle corresponds to the direction from the center of the marker toward the top edge, expressed in radians.


### Obstacle detection and Image processing
To detect obstacles, we first capture frames from the camera and apply a color filter using OpenCV’s `inRange` function (with a lower and upper threshold for blue) on the HSV representation of the image. This produces a mask, isolating only the blue pixels corresponding to our obstacles.

METTRE PHOTO DU MASK

Next, we apply a 3×3 Gaussian blur on the mask to reduce noise, followed by Canny edge detection to extract the contours of the polygonal obstacles.

METTRE SCREEN DU CANNY

With this edge image, we scale each obstacle’s contour using our custom `scale_contour` function, and then use the very powerful OpenCV functions `findContours `and `approxPolyDP` in our `detect_contours` function to recover the vertices of the scaled shapes. To avoid false positives, we restrict the obstacles to polygons with exactly four vertices.

Finally, we draw the scaled polygons in red, along with red circles indicating each detected vertex, on the output frame.


METTRE SCREEN DES SCALED POLYGONS